# Equity Visualization

ParkScore에 사용되는 지표 중 공평의 시각화를 진행한다.  
공평은 도시공원 배치의 "공간적 형평성"에 초점을 두어 점수화를 진행했다.  
해당 시각화는 85개 도시 중 "광주광역시"을 예로 들어 진행했다.

## Import

- 공공데이터 포털 : [전국도시공원정보표준데이터](https://www.data.go.kr/data/15012890/standard.do) 사용

In [1]:
!jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
!jupyter nbextension enable --sys-prefix --py pydeck

Installing /Users/kyeong6/anaconda3/lib/python3.11/site-packages/pydeck/nbextension/static -> pydeck
Removing: /Users/kyeong6/anaconda3/share/jupyter/nbextensions/pydeck
Symlinking: /Users/kyeong6/anaconda3/share/jupyter/nbextensions/pydeck -> /Users/kyeong6/anaconda3/lib/python3.11/site-packages/pydeck/nbextension/static
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable pydeck --py --sys-prefix
    
Enabling notebook extension pydeck/extensionRequires...
      - Validating: OK


In [2]:
import pandas as pd
import pydeck as pdk
import geopandas as gpd
from shapely import wkt
from shapely.geometry import MultiPolygon, Polygon, Point
import json

from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
try:
    df = pd.read_csv('/Users/kyeong6/Desktop/analysis/park/urban_park_data/urban_park.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('/Users/kyeong6/Desktop/analysis/park/urban_park_data/urban_park.csv', encoding='cp949')
    except Exception as e:
        print(f"Error: {e}")

In [4]:
df

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,41830-00017,광탄2호 소공원,소공원,NaN,경기도 양평군 용문면 광탄리 711,37.509628,127.628406,856.0,NaN,NaN,NaN,NaN,NaN,1994-02-25,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
1,41830-00031,광탄3호 소공원,소공원,NaN,경기도 양평군 용문면 광탄리 762,37.508443,127.627414,847.0,NaN,NaN,NaN,NaN,NaN,1994-02-25,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
2,41830-00018,공흥 소공원,소공원,NaN,경기도 양평군 양평읍 공흥리 885-1,37.493844,127.509326,1276.0,NaN,NaN,NaN,NaN,NaN,2012-11-22,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
3,41830-00019,한강아트로드 공원,소공원,NaN,경기도 양평군 강하면 운심리 43-25,37.496021,127.408216,3300.0,NaN,NaN,NaN,NaN,NaN,2012-08-30,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
4,41830-00020,강하 소공원,소공원,경기도 양평군 강하면 운심길 58,경기도 양평군 강하면 운심리 28-1,37.496164,127.412326,394.0,농구장+축구장+테니스장,NaN,NaN,NaN,NaN,2012-08-30,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18853,28110-00116,영종하늘도시 33호,근린공원,NaN,인천광역시 중구 중산동 1959-2,37.492194,126.575593,8652.0,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시 중구청,032-760-7777,2023-05-16,3490000,인천광역시 중구
18854,28110-00117,영종하늘도시 34호,근린공원,NaN,인천광역시 중구 운남동 1745-10,37.482082,126.540403,43514.0,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시 중구청,032-760-7777,2023-05-16,3490000,인천광역시 중구
18855,28110-00118,영종하늘도시 35호,근린공원,NaN,인천광역시 중구 운남동 1681-9,37.480161,126.509114,55128.0,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시 중구청,032-760-7777,2023-05-16,3490000,인천광역시 중구
18856,28110-00119,영종하늘도시 36호,근린공원,NaN,인천광역시 중구 중산동 1926-19,37.497137,126.575911,10172.0,NaN,NaN,NaN,NaN,NaN,NaN,인천광역시 중구청,032-760-7777,2023-05-16,3490000,인천광역시 중구


## Data Preprocessing

- "제공기관명"에서 광주광역시가 포함된 단어를 찾아 광주광역시 데이터를 얻는다.  

In [5]:
# "제공기관명" 열에서 "광주광역시"가 포함된 행 필터링
gwangju = df[df['제공기관명'].str.contains('광주광역시')]

In [6]:
gwangju 

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
79,29200-00173,선운,근린공원,광주광역시 광산구 선운로2번길 2-1,광주광역시 광산구 선암동 544외 4필지,35.148105,126.769051,59093.5,농구장+배드민턴장,NaN,화장실,NaN,관리사무소+노인복지관,2006-05-29,광주광역시 광산구청,062-960-8713,2023-03-16,3630000,광주광역시 광산구
80,29200-00174,우주,어린이공원,NaN,광주광역시 광산구 선암동 637,35.146419,126.775342,1509.1,NaN,시소+조합놀이,NaN,NaN,NaN,2006-05-29,광주광역시 광산구청,062-960-8689,2023-03-16,3630000,광주광역시 광산구
81,29200-00175,무지개,어린이공원,NaN,광주광역시 광산구 선암동 542,35.146506,126.778525,1585.0,NaN,그네+조합놀이,NaN,NaN,NaN,2006-05-29,광주광역시 광산구청,062-960-8689,2023-03-16,3630000,광주광역시 광산구
82,29200-00176,풍물,어린이공원,NaN,광주광역시 광산구 선암동 550,35.150287,126.776045,3388.5,운동공간,그네+시소+조합놀이+기타놀이기구,NaN,NaN,NaN,2006-05-29,광주광역시 광산구청,062-960-8689,2023-03-16,3630000,광주광역시 광산구
83,29200-00177,신창제2,어린이공원,NaN,광주광역시 광산구 신창동 산92-2,35.190883,126.848145,2000.0,NaN,NaN,NaN,NaN,NaN,2006-05-29,광주광역시 광산구청,062-960-8689,2023-03-16,3630000,광주광역시 광산구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18546,29170-00006,지우어린이공원,어린이공원,NaN,광주광역시 북구 운암동 478-1,35.175488,126.884076,3860.5,NaN,NaN,NaN,NaN,NaN,1970-06-05,광주광역시 북구청,062-410-6441,2024-01-31,3620000,광주광역시 북구
18547,29170-00007,신안어린이공원,어린이공원,NaN,광주광역시 북구 신안동 471-1,35.171617,126.898298,3817.5,NaN,NaN,NaN,NaN,NaN,1970-06-05,광주광역시 북구청,062-410-6441,2024-01-31,3620000,광주광역시 북구
18548,29170-00008,효동어린이공원,어린이공원,NaN,광주광역시 북구 중흥동 347-1,35.168287,126.909239,4205.0,NaN,NaN,NaN,NaN,NaN,1970-06-05,광주광역시 북구청,062-410-6441,2024-01-31,3620000,광주광역시 북구
18549,29170-00009,우산근린공원,근린공원,NaN,광주광역시 북구 우산동 228-1,35.175626,126.926242,133120.0,NaN,NaN,NaN,NaN,NaN,1975-02-18,광주광역시 북구청,062-410-6441,2024-01-31,3620000,광주광역시 북구


In [7]:
gwangju.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')

In [8]:
# 필요한 열만 사용
gwangju = gwangju[['공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적', '제공기관명']]
gwangju = gwangju.reset_index().drop(['index'], axis=1)

In [9]:
gwangju

,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,제공기관명
0,선운,근린공원,광주광역시 광산구 선운로2번길 2-1,광주광역시 광산구 선암동 544외 4필지,35.148105,126.769051,59093.5,광주광역시 광산구
1,우주,어린이공원,NaN,광주광역시 광산구 선암동 637,35.146419,126.775342,1509.1,광주광역시 광산구
2,무지개,어린이공원,NaN,광주광역시 광산구 선암동 542,35.146506,126.778525,1585.0,광주광역시 광산구
3,풍물,어린이공원,NaN,광주광역시 광산구 선암동 550,35.150287,126.776045,3388.5,광주광역시 광산구
4,신창제2,어린이공원,NaN,광주광역시 광산구 신창동 산92-2,35.190883,126.848145,2000.0,광주광역시 광산구
...,...,...,...,...,...,...,...,...
525,지우어린이공원,어린이공원,NaN,광주광역시 북구 운암동 478-1,35.175488,126.884076,3860.5,광주광역시 북구
526,신안어린이공원,어린이공원,NaN,광주광역시 북구 신안동 471-1,35.171617,126.898298,3817.5,광주광역시 북구
527,효동어린이공원,어린이공원,NaN,광주광역시 북구 중흥동 347-1,35.168287,126.909239,4205.0,광주광역시 북구
528,우산근린공원,근린공원,NaN,광주광역시 북구 우산동 228-1,35.175626,126.926242,133120.0,광주광역시 북구


### 결측치 해결

- 현재 소재지지번주소가 많은 값을 가지지만, 소재지지번주소가 NaN값이고 소재지도로명주소 값이 존재하는 경우가 있어 수작업으로 결측치 해결

In [10]:
# gwangju.to_csv('./gwangju.csv', encoding='cp949')

In [11]:
try:
    gwangju = pd.read_csv('./gwangju.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        gwangju = pd.read_csv('./gwangju.csv', encoding='cp949')
    except Exception as e:
        print(f"Error: {e}")

In [12]:
gwangju

,Unnamed: 0,소재지지번주소,latitude,longitude,urban_park
0,0,광주광역시 광산구 선암동 544외 4필지,35.148105,126.769051,59093.5
1,1,광주광역시 광산구 선암동 637,35.146419,126.775342,1509.1
2,2,광주광역시 광산구 선암동 542,35.146506,126.778525,1585.0
3,3,광주광역시 광산구 선암동 550,35.150287,126.776045,3388.5
4,4,광주광역시 광산구 신창동 산92-2,35.190883,126.848145,2000.0
...,...,...,...,...,...
525,525,광주광역시 북구 운암동 478-1,35.175488,126.884076,3860.5
526,526,광주광역시 북구 신안동 471-1,35.171617,126.898298,3817.5
527,527,광주광역시 북구 중흥동 347-1,35.168287,126.909239,4205.0
528,528,광주광역시 북구 우산동 228-1,35.175626,126.926242,133120.0


In [13]:
# # 필요하지 않은 열 제거
# gwangju = gwangju.drop(['공원명', '공원구분', '소재지도로명주소', '제공기관명'], axis=1)

### Geojson

- 광주광역시 행정구역(동) Geojson 사용  
- 해당 데이터는 qgis를 통해 shp 파일을 Geojson으로 변환

In [14]:
# geojson 업로드
geojson_file = './gwangju.geojson'
gwangju_gdf = gpd.read_file(geojson_file)

In [15]:
gwangju_gdf

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry
0,29110101,Daein-dong,대인동,"MULTIPOLYGON (((946602.933 1684771.874, 946606..."
1,29110102,Geumnamno 5(o)-ga,금남로5가,"MULTIPOLYGON (((946379.695 1684546.386, 946394..."
2,29110103,Chungjangno 5(o)-ga,충장로5가,"MULTIPOLYGON (((946229.498 1684396.631, 946229..."
3,29110104,Sugi-dong,수기동,"MULTIPOLYGON (((946147.252 1684316.603, 946151..."
4,29110105,Daeui-dong,대의동,"MULTIPOLYGON (((947208.394 1684087.413, 947208..."
...,...,...,...,...
196,29200175,Dongho-dong,동호동,"MULTIPOLYGON (((925390.284 1689048.240, 925421..."
197,29200176,Deongnim-dong,덕림동,"MULTIPOLYGON (((924553.149 1689031.013, 924576..."
198,29200177,Yangsan-dong,양산동,"MULTIPOLYGON (((926147.201 1689014.876, 926144..."
199,29200178,Dongnim-dong,동림동,"MULTIPOLYGON (((926413.305 1691165.365, 926415..."


In [16]:
# gwangju_gdf.to_csv('gwangju_gdf.csv', encoding='cp949')

### 데이터셋 변경

- 기존에 광주광역시 행정구역(동) Geojson 사용하였지만, 도시공원의 위도/경도가 존재하여 데이터셋을 변경 
- pydeck 코드에 적용하기 위해 열이름 변경 필요

In [17]:
gwangju

,Unnamed: 0,소재지지번주소,latitude,longitude,urban_park
0,0,광주광역시 광산구 선암동 544외 4필지,35.148105,126.769051,59093.5
1,1,광주광역시 광산구 선암동 637,35.146419,126.775342,1509.1
2,2,광주광역시 광산구 선암동 542,35.146506,126.778525,1585.0
3,3,광주광역시 광산구 선암동 550,35.150287,126.776045,3388.5
4,4,광주광역시 광산구 신창동 산92-2,35.190883,126.848145,2000.0
...,...,...,...,...,...
525,525,광주광역시 북구 운암동 478-1,35.175488,126.884076,3860.5
526,526,광주광역시 북구 신안동 471-1,35.171617,126.898298,3817.5
527,527,광주광역시 북구 중흥동 347-1,35.168287,126.909239,4205.0
528,528,광주광역시 북구 우산동 228-1,35.175626,126.926242,133120.0


In [18]:
# 열 이름 변경
gwangju.rename(columns={
    '위도' : 'latitude',
    '경도' : 'longitude',
    '공원면적' : 'urban_park'
}, inplace=True)

In [19]:
gwangju

,Unnamed: 0,소재지지번주소,latitude,longitude,urban_park
0,0,광주광역시 광산구 선암동 544외 4필지,35.148105,126.769051,59093.5
1,1,광주광역시 광산구 선암동 637,35.146419,126.775342,1509.1
2,2,광주광역시 광산구 선암동 542,35.146506,126.778525,1585.0
3,3,광주광역시 광산구 선암동 550,35.150287,126.776045,3388.5
4,4,광주광역시 광산구 신창동 산92-2,35.190883,126.848145,2000.0
...,...,...,...,...,...
525,525,광주광역시 북구 운암동 478-1,35.175488,126.884076,3860.5
526,526,광주광역시 북구 신안동 471-1,35.171617,126.898298,3817.5
527,527,광주광역시 북구 중흥동 347-1,35.168287,126.909239,4205.0
528,528,광주광역시 북구 우산동 228-1,35.175626,126.926242,133120.0


In [20]:
# gwangju.to_csv('/Users/kyeong6/desktop/gwangju.csv', encoding='cp949')

## Heatmap 

- gwangju_gdf.csv 파일에 "urban_park" 열을 생성하고, 도시공원 면적 값을 넣은 뒤 업로드

In [21]:
# Mapbox api token 설정
MAP_BOX_API = os.getenv('MAPBOX_API_KEY')
pdk.settings.mapbox_api_key = MAP_BOX_API

In [23]:
try:
    gwangju_gdf = pd.read_csv('./gwangju_gdf.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        gwangju_gdf = pd.read_csv('./gwangju_gdf.csv', encoding='cp949')
    except Exception as e:
        print(f"Error: {e}")

In [24]:
gwangju_gdf

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,urban_park,geometry
0,29110101,Daein-dong,대인동,424420.0,MULTIPOLYGON (((946602.933195839 1684771.87406...
1,29110102,Geumnamno 5(o)-ga,금남로5가,424420.0,MULTIPOLYGON (((946379.6951006013 1684546.3860...
2,29110103,Chungjangno 5(o)-ga,충장로5가,424420.0,MULTIPOLYGON (((946229.4983861081 1684396.6305...
3,29110104,Sugi-dong,수기동,424420.0,MULTIPOLYGON (((946147.2517786798 1684316.6025...
4,29110105,Daeui-dong,대의동,424420.0,MULTIPOLYGON (((947208.3938648928 1684087.4134...
...,...,...,...,...,...
196,29200175,Dongho-dong,동호동,53012.9,MULTIPOLYGON (((925390.2842348986 1689048.2395...
197,29200176,Deongnim-dong,덕림동,53012.9,MULTIPOLYGON (((924553.1492866804 1689031.0126...
198,29200177,Yangsan-dong,양산동,39684.0,MULTIPOLYGON (((926147.2005513686 1689014.8759...
199,29200178,Dongnim-dong,동림동,379395.9,MULTIPOLYGON (((926413.3045293004 1691165.3646...


In [25]:
# geometry 열을 shapely 객체로 변환
gwangju_gdf['geometry'] = gwangju_gdf['geometry'].apply(wkt.loads)

In [26]:
# # 폴리곤 및 멀티폴리곤 좌표 추출 함수 정의
# def polygon_to_coordinates(geom):
#     if isinstance(geom, Polygon):
#         lon, lat = geom.exterior.xy
#         return [[x, y] for x, y in zip(lon, lat)]
#     elif isinstance(geom, MultiPolygon):
#         coords = []
#         for poly in geom.geoms:
#             lon, lat = poly.exterior.xy
#             coords.append([[x, y] for x, y in zip(lon, lat)])
#         return coords
#     else:
#         return []

In [27]:
# # 좌표 변환 함수 적용
# gwangju_gdf['coordinates'] = gwangju_gdf['geometry'].apply(polygon_to_coordinates)

In [28]:
gwangju_gdf

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,urban_park,geometry
0,29110101,Daein-dong,대인동,424420.0,MULTIPOLYGON (((946602.933195839 1684771.87406...
1,29110102,Geumnamno 5(o)-ga,금남로5가,424420.0,MULTIPOLYGON (((946379.6951006013 1684546.3860...
2,29110103,Chungjangno 5(o)-ga,충장로5가,424420.0,MULTIPOLYGON (((946229.4983861081 1684396.6305...
3,29110104,Sugi-dong,수기동,424420.0,MULTIPOLYGON (((946147.2517786798 1684316.6025...
4,29110105,Daeui-dong,대의동,424420.0,MULTIPOLYGON (((947208.3938648928 1684087.4134...
...,...,...,...,...,...
196,29200175,Dongho-dong,동호동,53012.9,MULTIPOLYGON (((925390.2842348986 1689048.2395...
197,29200176,Deongnim-dong,덕림동,53012.9,MULTIPOLYGON (((924553.1492866804 1689031.0126...
198,29200177,Yangsan-dong,양산동,39684.0,MULTIPOLYGON (((926147.2005513686 1689014.8759...
199,29200178,Dongnim-dong,동림동,379395.9,MULTIPOLYGON (((926413.3045293004 1691165.3646...


In [29]:
gwangju_gdf['geometry'].dtype

dtype('O')

In [30]:
# GeoDataFrame 생성
gwangju_gdf = gpd.GeoDataFrame(gwangju_gdf, geometry='geometry')

# 현재 좌표계를 설정
gwangju_gdf.crs = "EPSG:5179"

# 좌표계 WGS84로 변환
gwangju_gdf = gwangju_gdf.to_crs(epsg=4326)

In [31]:
# 중심 좌표 계산 함수
def calculate_centroid(geometry):
    if isinstance(geometry, MultiPolygon):
        centroid = geometry.centroid
    elif isinstance(geometry, Polygon):
        centroid = geometry.centroid
    return centroid.y, centroid.x

In [32]:
# 중심 좌표 구하기(위도 / 경도)
gwangju_gdf['latitude'], gwangju_gdf['longitude'] = zip(*gwangju_gdf['geometry'].apply(calculate_centroid))

In [33]:
gwangju_gdf['latitude'].dtype

dtype('float64')

In [34]:
#  필요한 열만 선택
gwangju_gdf = gwangju_gdf[['EMD_KOR_NM', 'urban_park', 'latitude', 'longitude']]

In [35]:
# null 값 확인
print(gwangju_gdf.isnull().sum())

EMD_KOR_NM    0
urban_park    0
latitude      0
longitude     0
dtype: int64


In [36]:
gwangju_gdf

,EMD_KOR_NM,urban_park,latitude,longitude
0,대인동,424420.0,35.154010,126.915223
1,금남로5가,424420.0,35.152697,126.912337
2,충장로5가,424420.0,35.151344,126.911238
3,수기동,424420.0,35.150502,126.909713
4,대의동,424420.0,35.149117,126.920520
...,...,...,...,...
196,동호동,53012.9,35.182639,126.687332
197,덕림동,53012.9,35.183131,126.670041
198,양산동,39684.0,35.197487,126.688780
199,동림동,379395.9,35.206535,126.691207


In [37]:
# HeatmapLayer 생성
layer = pdk.Layer(
    'HeatmapLayer',
    data=gwangju_gdf,
    get_position='[longitude, latitude]',
    get_weight='urban_park',
    radiusPixels=50,
)

# pydeck View 생성
view_state = pdk.ViewState(
    longitude=gwangju_gdf['longitude'].mean(),
    latitude=gwangju_gdf['latitude'].mean(),
    zoom=12
)

# pydeck Deck 생성
deck = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
#     map_style='mapbox://styles/mapbox/light-v10',  # Mapbox 스타일 설정
    tooltip={"text": "동: {EMD_KOR_NM}\n도시공원 면적: {urban_park} sqm"}
)

In [38]:
# pydeck 저장 및 표시
output_path = './equity_html/equity_heatmap.html'
deck.to_html(output_path)

deck.show()

## 데이터셋 변경 후 Heatmap

- 기존 데이터셋으로는 heatmap의 범위가 제대로 파악되지 않아 도시공원 위도 / 경도를 이용한 heatmap 확인
- heatmap에서의 도시의 경계가 명확하지 않아 Polygon layer를 추가

In [39]:
# Mapbox api token 설정
MAP_BOX_API = os.getenv('MAPBOX_API_KEY')
pdk.settings.mapbox_api_key = MAP_BOX_API

In [40]:
try:
    gwangju = pd.read_csv('./gwangju.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        gwangju = pd.read_csv('./gwangju.csv', encoding='cp949')
    except Exception as e:
        print(f"Error: {e}")

In [41]:
gwangju = gwangju.drop(['Unnamed: 0'], axis=1)

In [42]:
gwangju

,소재지지번주소,latitude,longitude,urban_park
0,광주광역시 광산구 선암동 544외 4필지,35.148105,126.769051,59093.5
1,광주광역시 광산구 선암동 637,35.146419,126.775342,1509.1
2,광주광역시 광산구 선암동 542,35.146506,126.778525,1585.0
3,광주광역시 광산구 선암동 550,35.150287,126.776045,3388.5
4,광주광역시 광산구 신창동 산92-2,35.190883,126.848145,2000.0
...,...,...,...,...
525,광주광역시 북구 운암동 478-1,35.175488,126.884076,3860.5
526,광주광역시 북구 신안동 471-1,35.171617,126.898298,3817.5
527,광주광역시 북구 중흥동 347-1,35.168287,126.909239,4205.0
528,광주광역시 북구 우산동 228-1,35.175626,126.926242,133120.0


In [43]:
# 필요한 열만 선택
gwangju = gwangju[['urban_park', 'latitude', 'longitude']]

In [44]:
# null 값 확인
print(gwangju.isnull().sum())

urban_park    0
latitude      0
longitude     0
dtype: int64


In [45]:
gwangju

,urban_park,latitude,longitude
0,59093.5,35.148105,126.769051
1,1509.1,35.146419,126.775342
2,1585.0,35.146506,126.778525
3,3388.5,35.150287,126.776045
4,2000.0,35.190883,126.848145
...,...,...,...
525,3860.5,35.175488,126.884076
526,3817.5,35.171617,126.898298
527,4205.0,35.168287,126.909239
528,133120.0,35.175626,126.926242


In [46]:
# HeatmapLayer 생성
heatmap_layer = pdk.Layer(
    'HeatmapLayer',
    data=gwangju,
    get_position='[longitude, latitude]',
    get_weight='urban_park',
    radiusPixels=50,
)

In [47]:
# pydeck View 생성
view_state = pdk.ViewState(
    longitude=gwangju['longitude'].mean(),
    latitude=gwangju['latitude'].mean(),
    zoom=12
)

In [48]:
# pydeck Deck을 생성
deck = pdk.Deck(
    layers=heatmap_layer,
    initial_view_state=view_state,
#     map_style='mapbox://styles/mapbox/light-v10'
    tooltip={"text": "도시공원 면적: {urban_park} sqm"}
)

In [49]:
# pydeck 저장 및 표시
output_path = './equity_html/equity_heatmap2.html'
deck.to_html(output_path)

deck.show()